In [1]:
import os

In [2]:
%pwd

'/Users/vatsalkhetan/Downloads/Chest_CT_Cancer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vatsalkhetan/Downloads/Chest_CT_Cancer'

In [35]:
from dataclasses import dataclass
from pathlib import Path




@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_classes: int
    params_weights: str
    params_learning_rate: int

In [36]:
from cnnclassifer.constants import *
from cnnclassifer.utils.common import read_yaml, create_directories

In [42]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_weights=params.WEIGHTS,
            params_classes=params.CLASSES,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config

In [43]:
import os
import urllib.request as request
from zipfile import ZipFile
import torch
import torch.nn as nn
import torchvision.models as models
import time

In [44]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from pathlib import Path

class Training:
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.get_base_model()
        # Load the base model from the specified path
        
    def get_base_model(self):
        # Load a VGG16 model without pretrained weights
        self.model = models.vgg16(pretrained=(self.config.params_weights == "imagenet"))
        
        # Modify the final layer of the classifier to match the number of classes
        in_features = self.model.classifier[-1].in_features  # Get the input features of the last layer
        self.model.classifier[-1] = nn.Linear(in_features, self.config.params_classes)  # New output layer

        # Load the state_dict if available, with strict=False to handle any structural differences
        if Path(self.config.updated_base_model_path).exists():
            self.model.load_state_dict(torch.load(self.config.updated_base_model_path), strict=False)

        # Move the model to the specified device (GPU or CPU)
        self.model = self.model.to(self.device)

    def train_valid_generator(self):
        # Define transformations for data augmentation and normalization
        train_transform = transforms.Compose([
            transforms.RandomRotation(40),
            transforms.RandomHorizontalFlip(),
            transforms.RandomResizedCrop(self.config.params_image_size[0]),
            transforms.RandomAffine(degrees=0, shear=0.2),
            transforms.RandomResizedCrop(self.config.params_image_size[0], scale=(0.8, 1.2)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]) if self.config.params_is_augmentation else transforms.Compose([
            transforms.Resize(self.config.params_image_size[:-1]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        valid_transform = transforms.Compose([
            transforms.Resize(self.config.params_image_size[:-1]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        # Load training and validation data
        self.train_dataset = datasets.ImageFolder(root=self.config.training_data, transform=train_transform)
        self.valid_dataset = datasets.ImageFolder(root=self.config.training_data, transform=valid_transform)

        # Create data loaders
        self.train_loader = DataLoader(self.train_dataset, batch_size=self.config.params_batch_size, shuffle=True)
        self.valid_loader = DataLoader(self.valid_dataset, batch_size=self.config.params_batch_size, shuffle=False)

    def train(self):
        # Set up the training parameters
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.model.parameters(), lr=self.config.params_learning_rate)
        num_epochs = self.config.params_epochs

        for epoch in range(num_epochs):
            # Training phase
            self.model.train()
            running_loss = 0.0
            for images, labels in self.train_loader:
                images, labels = images.to(self.device), labels.to(self.device)

                optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * images.size(0)

            epoch_loss = running_loss / len(self.train_loader.dataset)
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

            # Validation phase
            self.model.eval()
            val_loss = 0.0
            correct = 0
            with torch.no_grad():
                for images, labels in self.valid_loader:
                    images, labels = images.to(self.device), labels.to(self.device)
                    outputs = self.model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * images.size(0)
                    _, preds = torch.max(outputs, 1)
                    correct += torch.sum(preds == labels).item()

            val_loss /= len(self.valid_loader.dataset)
            val_accuracy = correct / len(self.valid_loader.dataset)
            print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

        # Save the trained model
        self.save_model(path=self.config.trained_model_path, model=self.model)

    @staticmethod
    def save_model(path: Path, model):
        torch.save(model.state_dict(), path)


In [45]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-11-07 17:46:58,086: INFO: common: YAML file '/Users/vatsalkhetan/Downloads/Chest_CT_Cancer/config/config.yaml' loaded successfully.]
Loaded YAML content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}}
Type of content: <class 'dict'>
[2024-11-07 17:46:58,089: INFO: common: YAML file '/Users/vatsalkhetan/Downloads/Chest_CT_Cancer/params.yaml' loaded successfully.]
Loaded YAML content: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3],

/Users/vatsalkhetan/miniconda3/envs/cancer/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vatsalkhetan/miniconda3/envs/cancer/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/5], Loss: 0.5138
Validation Loss: 0.2684, Accuracy: 0.9184
Epoch [2/5], Loss: 0.1574
Validation Loss: 0.1314, Accuracy: 0.9446
Epoch [3/5], Loss: 0.1152
Validation Loss: 0.3551, Accuracy: 0.9359
Epoch [4/5], Loss: 0.1990
Validation Loss: 0.0994, Accuracy: 0.9504
Epoch [5/5], Loss: 0.1185
Validation Loss: 0.1018, Accuracy: 0.9708
